<a href="https://colab.research.google.com/github/jjangmo91/ParkLab/blob/main/project/2025%20NASA%20Space%20Apps%20Challenge/02_Tonle_sap_Lake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap

# Earth Engine 인증
ee.Authenticate()
# Earth Engine 초기화
ee.Initialize(project='ee-jjangmo91') # 본인 계정

In [ ]:
# ==============================================================================
# 0단계: 지도 생성 및 AOI 그리기 (이 부분은 수동으로 먼저 실행)
# ==============================================================================
# 지도를 먼저 생성하고, 왼쪽의 다각형(polygon) 도구로 원하는 영역을 그립니다.
m = geemap.Map(center=[12.7, 104.3], zoom=9)
m.add_basemap('HYBRID')
display(m) # AOI를 그립니다.

In [ ]:
# ==============================================================================
# 1단계: 분석 파라미터 설정
# ==============================================================================
# 이 코드를 실행하기 전에 반드시 위 지도에서 AOI 그리기를 완료해야 합니다.
aoi_tonlesap = m.user_roi

# SAR 후방산란계수 임계값 (dB)
water_threshold = -16

# 분석할 연도
baseline_years = [2017, 2018]
drought_years = [2019, 2020]

# 분석할 계절을 여기서 쉽게 변경
start_md = '12-01' # 시작 월-일
end_md = '03-31'   # 종료 월-일

# ==============================================================================
# 2단계: 분석 함수 정의 ('변화 탐지' 로직)
# ==============================================================================
def create_water_mask(year, aoi, start_month_day, end_month_day):
    """지정된 연도와 기간의 수역 마스크를 생성하는 범용 함수"""
    start_month = int(start_month_day.split('-')[0])
    end_month = int(end_month_day.split('-')[0])

    # 계절이 연도를 넘어가는 경우 (예: 12월-3월)
    if start_month > end_month:
        start_date = f'{year-1}-{start_month_day}'
        end_date = f'{year}-{end_month_day}'
    # 계절이 같은 해에 끝나는 경우 (예: 8월-10월)
    else:
        start_date = f'{year}-{start_month_day}'
        end_date = f'{year}-{end_month_day}'

    s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi).filterDate(start_date, end_date) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')).select('VV')

    # 데이터가 없는 경우를 대비한 안전장치
    image_count = s1_collection.size()
    water_mask = ee.Image(ee.Algorithms.If(
        image_count.gt(0),
        s1_collection.min().clip(aoi).lt(water_threshold).selfMask() \
            .connectedPixelCount(100, False).gte(100).selfMask(),
        ee.Image(0).selfMask() # 데이터가 없으면 빈 이미지 반환
    ))
    return water_mask

# ==============================================================================
# 3단계: '변화 탐지' 로직으로 면적 계산
# ==============================================================================
print("변화 탐지 기반 수역 면적 분석을 시작합니다 (기간 통일: 12월-3월)...")
print("--> '평년' 기준 수역을 생성합니다...")
baseline_masks = [create_water_mask(year, aoi_tonlesap, start_md, end_md) for year in baseline_years]
baseline_mask = ee.ImageCollection(baseline_masks).max().selfMask()
baseline_area = calculate_area(baseline_mask, aoi_tonlesap)
print(f"💧 평년({baseline_years[0]}-{baseline_years[1]}) 기준 수역 면적: {baseline_area.getInfo():,.2f} km²")

drought_results = {}
for year in drought_years:
    print(f"--> {year}년 가뭄 영향을 분석합니다...")
    drought_mask_raw = create_water_mask(year, aoi_tonlesap, start_md, end_md)
    persistent_water = baseline_mask.And(drought_mask_raw).selfMask()
    persistent_area = calculate_area(persistent_water, aoi_tonlesap)
    percent_change = persistent_area.divide(baseline_area).subtract(1).multiply(100)
    drought_results[year] = {'mask': persistent_water, 'area': persistent_area}
    print(f"🔥 {year}년 잔존 수역 면적: {persistent_area.getInfo():,.2f} km² (평년 대비 {percent_change.getInfo():.2f}%)")

# ==============================================================================
# 4단계: 지도 시각화
# ==============================================================================
result_map = geemap.Map(center=m.center, zoom=m.zoom)
result_map.add_basemap('HYBRID')
result_map.addLayer(aoi_tonlesap, {'color': 'grey'}, '분석 영역 (AOI)')
lost_water_2020 = baseline_mask.And(drought_results[2020]['mask'].unmask().Not()).selfMask()
result_map.addLayer(baseline_mask, {'palette': '#87CEEB'}, f'평년({baseline_years[0]}-{baseline_years[1]}) 기준 수역')
result_map.addLayer(drought_results[2020]['mask'], {'palette': '#00008B'}, '2020년 잔존 수역 (진한 파랑)')
result_map.addLayer(lost_water_2020, {'palette': '#FF4500'}, '2020년 증발 수역 (주황)')
result_map.add_legend(
    title="2020년 가뭄 영향 (12월-3월)",
    legend_dict={
        '분석 영역 (AOI)': '808080',
        '평년 기준 수역': '87CEEB',
        '가뭄에도 남은 물': '00008B',
        '가뭄으로 사라진 물': 'FF4500'
    },
)
result_map.addLayerControl()
display(result_map)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# ==============================================================================
# 1-1: 시계열 분석 파라미터 설정
# ==============================================================================
# 분석할 전체 연도 범위 설정
start_year = 2015
end_year = 2024

start_md = '12-01' # 건기 시작 월-일
end_md = '03-31'   # 건기 종료 월-일

# 이전에 정의된 분석 함수들을 그대로 사용합니다.
# create_water_mask, calculate_area, aoi_tonlesap

# ==============================================================================
# 1-2: '변화 탐지' 기반 연도별 건기 면적 계산
# ==============================================================================
print(f"변화 탐지 기반 건기 수역 시계열 분석을 시작합니다 ({start_year}-{end_year})...")

# 먼저, 비교의 기준이 될 '평년 기준선' 마스크와 면적을 계산합니다.
print("--> '평년(2017-2018)' 기준 수역을 생성합니다...")
# [수정] 함수에 start_md, end_md 변수 전달
baseline_masks = [create_water_mask(year, aoi_tonlesap, start_md, end_md) for year in [2017, 2018]]
baseline_mask = ee.ImageCollection(baseline_masks).max().selfMask()
baseline_area = calculate_area(baseline_mask, aoi_tonlesap)
print(f"    기준이 되는 평년 수역 면적: {baseline_area.getInfo():,.2f} km²")

# 이제 전체 기간에 대해 '변화 탐지' 로직을 적용합니다.
time_series_results = []
for year in range(start_year, end_year + 1):
    print(f"--> {year}년 건기 분석 중...")
    try:
        # 해당 연도의 원본 수역 마스크 생성
        # [수정] 함수에 start_md, end_md 변수 전달
        current_year_mask = create_water_mask(year, aoi_tonlesap, start_md, end_md)

        # [핵심 논리] 평년 기준선 마스크와 겹치는 부분만 '잔존 수역'으로 인정
        persistent_water = baseline_mask.And(current_year_mask).selfMask()

        # '잔존 수역'의 면적 계산
        persistent_area = calculate_area(persistent_water, aoi_tonlesap)
        area_km2 = persistent_area.getInfo()

        if area_km2 > 0:
            time_series_results.append({'year': year, 'area_km2': area_km2})
            print(f"    {year}년 잔존 수역 면적: {area_km2:,.2f} km²")
        else:
            print(f"    {year}년 데이터 없음.")

    except Exception as e:
        print(f"    {year}년 분석 중 오류 발생: 데이터가 부족할 수 있습니다.")

# ==============================================================================
# 1-3: 결과 시각화 (막대그래프) - 변경 없음
# ==============================================================================
if time_series_results:
    df_ts = pd.DataFrame(time_series_results)
    baseline_area_val = baseline_area.getInfo()

    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 6))

    bars = ax.bar(df_ts['year'], df_ts['area_km2'], color='skyblue')

    ax.axhline(y=baseline_area_val, color='red', linestyle='--', linewidth=2,
               label=f'Baseline Area (2017-18 Avg): {baseline_area_val:,.2f} km²')

    for bar in bars:
        if bar.get_height() < baseline_area_val:
            bar.set_color('salmon')

    ax.set_title(f'Tonle Sap Lake: Annual Persistent Water Area in Dry Season ({start_year}-{end_year})', fontsize=16)
    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Persistent Water Area (km²)', fontsize=12)

    ax.set_xticks(df_ts['year'])
    ax.tick_params(axis='x', rotation=45)
    ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
    ax.legend()
    plt.tight_layout()
    plt.show()
else:
    print("분석할 데이터가 없어 그래프를 생성할 수 없습니다.")